# **Previsão de Pouso do Primeiro Estágio de Foguetes Falcon 9**


### Coleta de dados utilizando Web Scraping

Nesta seção, será utilizado o web scraping (processo de extração de dados de uma página da web) para coletar os dados dos registros de lançamentos dos foguetes Falcon 9, localizados na página do Wikipedia, com seguinte URL:

[https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches](https://en.wikipedia.org/wiki/List_of_Falcon\_9\_and_Falcon_Heavy_launches?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/Falcon9\_rocket_family.svg)


Os registros dos lançamentos dos foguetes da SpaceX estão armazenados em uma tabela HTML, conforme apresentado a seguir:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_1\_L2/images/falcon9-launches-wiki.png)


### Objetivos

*   Extrair (webscrape) os registros dos lançamentos do foguete Falcon 9 de uma tabela HTML do Wikipedia usando `BeautifulSoup`;
*   Processar a tabela HTML e converter em um dataframe pandas.


### Instalação e importação das bibliotecas/pacotes necessários


In [2]:
# Instalação do pacote beautifulsoup
!pip3 install beautifulsoup4
#!pip3 install requests

You should consider upgrading via the 'c:\users\oliveira\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 97.4/97.4 KB 1.4 MB/s eta 0:00:00


In [3]:
# Importação dos pacotes necessários
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

### Definição de funções auxiliares para processamento da tabela HTML extraída


In [4]:
# Retorna os dados das células referentes à data e hora da tabela HTML
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

# Retorna os dados das células referentes à booster version da tabela HTML
def booster_version(table_cells):
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

# Retorna os dados das células referentes à landing status da tabela HTML
def landing_status(table_cells):
    out=[i for i in table_cells.strings][0]
    return out

# Retorna os dados das células referentes à massa dos foguetes (payload mass) da tabela HTML
def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

# Extrai o nome da coluna a partir de seu cabeçalho
def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    colunm_name = ' '.join(row.contents)
    # Filtra dígitos e nomes vazios
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


Obs. Para tornar todo o processo mais consistente, utilizaremos um url estático do wikipedia da `Lista dos lançamentos dos foguetes Falcon 9 e Falcon Heavy`, atualizado a ultima vez no dia `9 Junho 2021`.


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### Solicitação da página Wikipedia dos lançamentos Falcon9 a partir do seu URL

In [13]:
# Usa o método requests.get() com o static_url e atribui a resposta da página à um objeto 'response'
response = requests.get(static_url)

In [14]:
# Usa BeautifulSoup() para criar um objeto BeautifulSoup a partir da resposta (response) de conteúdo textual
soup = BeautifulSoup(response.content, 'html.parser')

In [17]:
# Usa o atributo soup.title para verificar se o título da página do objeto BeautifulSoup criado corresponde ao título da página propriamente dita
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### Extração dos nomes das colunas/variáveis a partir do cabeçalho da tabela HTML


In [20]:
# Usa a função find_all no objeto BeautifulSoup, com o elemento do tipo `table`, para procurar todas as tabelas presentes na página
# Atribui o resultado à uma lista `html_tables` contendo todas as tabelas encontradas na página 
html_tables = soup.find_all('table')

In [21]:
# Verificação de qual elemento da lista corresponde à tabela contendo os dados de lançamentos dos foguetes Falcon 9
# No caso, o terceiro elemento da lista (índice 2)
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

Os nomes das colunas da tabela podem ser identificados pelo prefixo `<th>` do cabeçalho da tabela HTML conforme a seguir:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Extração dos nomes das colunas através da iteração sobre os elementos `<th>` da tabela HTML, aplicando a função `extract_column_from_header()`:


In [25]:
column_names = []

# Aplica a função find_all() usando o elemento `th` na tabela 'first_launch_table'
# Itera sobre cada elemento `th` e aplica a função 'extract_column_from_header()' para extrair o nome da coluna
# Anexa o nome das colunas não vazias (`se o nome não é None e o len(name) > 0`) em uma lista chamada 'column_names'

for x in range(len(first_launch_table.find_all('th'))):
    try:
        name = extract_column_from_header(first_launch_table.find_all('th')[x])
        if (name is not None and len(name) > 0):
            column_names.append(name)
    except:
        pass

In [26]:
# Apresenta os nomes das colunas extraídos da tabela HTML
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


### Criação de um dataframe pelo processamento da tabela HTML dos lançamentos

Criação de um dicionário vazio contendo keys correspondentes ao nomes das colunas extraídos. 

In [27]:
launch_dict= dict.fromkeys(column_names)

# Remove colunas irrelevantes para o dataframe
del launch_dict['Date and time ( )']

# Cria um dicionário 'launch_dict' com cada value sendo uma lista vazia
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Adição de algumas novas colunas para informações extras
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

A seguir, preencheremos as listas vazias do dicionário `launch_dict` com os registros dos lançamentos extraídos das linhas da tabela HTML


Obs. Tabelas HTML das paginas Wiki gerlamente contém valores não esperados como links de referência `B0004.1[8]`, valores ausentes `N/A [e]`, formatos inconsistentes, etc


In [41]:
extracted_row = 0
# Extrai cada tabela
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # Obtém a linha da tabela 
    for rows in table.find_all("tr"):
        # Verifica se o primeiro cabeçalho da tabela corresponde ao número de um lançamento 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        # Obtém o elemento 'td' da tabela 
        row=rows.find_all('td')
        # Se for um número salva as células em um dicionário 
        if flag:
            # Valor de Flight Number
            extracted_row += 1
            # Anexa o número do voo da tabela na key `Flight No.` do dicionário launch_dict
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date
            # Anexa a data da tabela na key `Date` do dicionário launch_dict
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Time
            # Anexa o tempo da tabela na key `Date` do dicionário launch_dict
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            # Anexa o booster version (bv) da tabela na key `Version Booster` do dicionário launch_dict
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            # Anexa o launch_site da tabela na key `Launch site` do dicionário launch_dict
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)
            
            # Payload
            # Anexa o payload da tabela na key `Payload` do dicionário launch_dict
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)
            
            # Payload Mass
            # Anexa o payload_mass da tabela na key `Payload mass` do dicionário launch_dict
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)
            
            # Orbit
            # Anexa o orbit da tabela na key `Orbit` do dicionário launch_dict
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)
            
            # Customer
            # Anexa o customer da tabela na key `Customer` do dicionário launch_dict
            customer = row[6].text.strip()
            launch_dict["Customer"].append(customer)
            #print(customer)
            
            # Launch outcome
            # Anexa o launch_outcome da tabela na key `Launch outcome` do dicionário launch_dict
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            
            # Booster landing
            # Anexa o booster_landing da tabela na key `Booster landing` do dicionário launch_dict
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)
            

F9 v1.0B0003.1
F9 v1.0B0004.1
F9 v1.0B0005.1
F9 v1.0B0006.1
F9 v1.0B0007.1
F9 v1.1B1003
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 v1.1
F9 FT
F9 v1.1
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT
F9 FT♺
F9 FT
F9 FT
F9 FT
F9 FTB1029.2
F9 FT
F9 FT
F9 B4
F9 FT
F9 B4
F9 B4
F9 FTB1031.2
F9 B4
F9 FTB1035.2
F9 FTB1036.2
F9 B4
F9 FTB1032.2
F9 FTB1038.2
F9 B4
F9 B4B1041.2
F9 B4B1039.2
F9 B4
F9 B5B1046.1
F9 B4B1043.2
F9 B4B1040.2
F9 B4B1045.2
F9 B5
F9 B5B1048
F9 B5B1046.2
F9 B5
F9 B5B1048.2
F9 B5B1047.2
F9 B5B1046.3
F9 B5
F9 B5
F9 B5B1049.2
F9 B5B1048.3
F9 B5[268]
F9 B5
F9 B5B1049.3
F9 B5B1051.2
F9 B5B1056.2
F9 B5B1047.3
F9 B5
F9 B5
F9 B5B1056.3
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5B1058.2
F9 B5
F9 B5B1049.6
F9 B5
F9 B5B1060.2
F9 B5B1058.3
F9 B5B1051.6
F9 B5
F9 B5
F9 B5
F9 B5
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5
F9 B5B1051.8
F9 B5B1058.5
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5 ♺
F9 B5B1060.6
F9

Após o preenchimento dos valores em launch_dict, pode-se criar o dataframe


In [43]:
# Transforma o dicionario em dataframe
df=pd.DataFrame(launch_dict)
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Success\n,4 June 2010,18:45
1,1,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0003.1,Success,4 June 2010,18:45
2,1,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0003.1,Success,4 June 2010,18:45
3,1,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0003.1,Success\n,4 June 2010,18:45
4,1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0004.1,Success\n,4 June 2010,15:43
...,...,...,...,...,...,...,...,...,...,...,...
545,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1051.10,Success,9 May 2021,06:42
546,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX Capella Space and Tyvak,Success\n,F9 B5B1058.8,Success,15 May 2021,22:56
547,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1063.2,Success,26 May 2021,18:59
548,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA (CRS),Success\n,F9 B5B1067.1,Success,3 June 2021,17:29


### Exportação do dataframe final para o formato CSV


In [44]:
df.to_csv('spacex_web_scraped.csv', index=False)